In [ ]:
import ray 

ray.init(num_gpus=1)

In [ ]:
import ray
config = {
    "env": "Pendulum-v1",
    "num_gpus":1,
}
stop = {"episode_reward_mean": -1000}
ray.shutdown()
ray.init(
  num_gpus=1,
  include_dashboard=False,
  ignore_reinit_error=True,
  log_to_driver=False,
)
# execute training 
analysis = ray.tune.run(
  "SAC",
  config=config,
  stop=stop,
  checkpoint_at_end=True,
  local_dir="Pendulum_v1"  
)

In [ ]:
import ray.rllib.algorithms.sac as sac
# restore a trainer from the last checkpoint
trial = analysis.get_best_logdir("episode_reward_mean", "max")
checkpoint = analysis.get_best_checkpoint(
  trial,
  "training_iteration",
  "max",
)
trainer = sac.SAC(config=config)
trainer.restore(checkpoint)

In [ ]:
from gym.wrappers import RecordVideo

env = RecordVideo(gym.make("Pendulum-v1"),"sac_video")
obs = env.reset()

while True:
    action = trainer.compute_action(obs)
    obs,reward,done, _ = env.step(action)
    if done:
        break
env.close()